<a href="https://colab.research.google.com/github/pallavi-allada/experiments-with-data/blob/master/ObjectDetection_MaskRCNN_Keras_Matterport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download the matterport repo 

In [1]:
!git clone https://github.com/matterport/Mask_RCNN.git

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 111.90 MiB | 25.71 MiB/s, done.
Resolving deltas: 100% (571/571), done.


In [0]:
import os
os.chdir('/content/Mask_RCNN')

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Install Matterport implementation of Mask RCNN

In [4]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating mask_rcnn.egg-info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/mrcnn
copying mrcnn/model.py -> build/lib/mrcnn
copying mrcnn/visualize.py -> build/lib/mrcnn
copying mrcnn/parallel_model.py -> build/lib/mrcnn
copying mrcnn/config.py -> build/lib/mrcnn
copying mrcnn/utils.py -> build/lib/mrcnn
copying mrcnn/__init__.py -> build/lib/mrcnn
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/model.py -> build/bdist.linux-x86_64/egg/mrcnn

In [5]:
# import the necessary packages
from mrcnn.config import Config
from mrcnn import model as modellib
from mrcnn import visualize
import numpy as np
import colorsys
import argparse
import imutils
import random
import cv2
import os

Using TensorFlow backend.


The mrcnn  imports are from Matterport’s implementation of Mask R-CNN. From mrcnn , we’ll use Config  to create a custom subclass for our configuration, modellib  to load our model, and visualize  to draw our masks.

In [0]:
import pathlib
from pathlib import Path
LBLS_FILE = '/content/drive/My Drive/Colab Notebooks/ObjectDetection/coco_labels.txt'
WTS_FILE = '/content/drive/My Drive/Colab Notebooks/ObjectDetection/mask_rcnn_coco.h5'
IMG_FILE_DIR = '/content/drive/My Drive/Colab Notebooks/ObjectDetection/images'

In [0]:
# load the class label names from disk, one label per line
CLASS_NAMES = open(LBLS_FILE).read().strip().split("\n")

In [0]:
CLASS_NAMES

In [0]:
# generate random (but visually distinct) colors for each class label
# (thanks to Matterport Mask R-CNN for the method!)
hsv = [(i / len(CLASS_NAMES), 1, 1.0) for i in range(len(CLASS_NAMES))]
COLORS = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
random.seed(42)
random.shuffle(COLORS)


In [0]:
class SimpleConfig(Config):
	# give the configuration a recognizable name
	NAME = "coco_inference"

	# set the number of GPUs to use along with the number of images
	# per GPU
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1

	# number of classes (we would normally add +1 for the background
	# but the background class is *already* included in the class
	# names)
	NUM_CLASSES = len(CLASS_NAMES)

In [10]:
# initialize the inference configuration
config = SimpleConfig()

# initialize the Mask R-CNN model for inference and then load the
# weights
print("[INFO] loading Mask R-CNN model...")
model = modellib.MaskRCNN(mode="inference", config=config,
	model_dir=os.getcwd())
model.load_weights(WTS_FILE, by_name=True)

W0820 13:07:14.945596 139635400853376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0820 13:07:15.001419 139635400853376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0820 13:07:15.008118 139635400853376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0820 13:07:15.053613 139635400853376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1919: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0820 13:07:15.058045 139635400

[INFO] loading Mask R-CNN model...


W0820 13:07:18.568777 139635400853376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2018: The name tf.image.resize_nearest_neighbor is deprecated. Please use tf.compat.v1.image.resize_nearest_neighbor instead.

W0820 13:07:19.185789 139635400853376 deprecation_wrapper.py:119] From /content/Mask_RCNN/mrcnn/model.py:341: The name tf.log is deprecated. Please use tf.math.log instead.

W0820 13:07:19.205361 139635400853376 deprecation.py:323] From /content/Mask_RCNN/mrcnn/model.py:399: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0820 13:07:19.217811 139635400853376 deprecation.py:506] From /content/Mask_RCNN/mrcnn/model.py:423: calling crop_and_resize_v1 (from tensorflow.python.ops.image_ops_impl) with box_ind is deprecated and will be removed in a 

In [0]:
import os
import random
import shutil

IMG_FILE_DIR = '/content/drive/My Drive/Colab Notebooks/ObjectDetection/images/classification files/tufted_headboards'
KITCHEN_DIR = '/content/drive/My Drive/Colab Notebooks/ObjectDetection/images/detected_objects/kitchen'
BEDROOM_DIR = '/content/drive/My Drive/Colab Notebooks/ObjectDetection/images/detected_objects/bedroom'
LIVING_DIR = '/content/drive/My Drive/Colab Notebooks/ObjectDetection/images/detected_objects/living'
PASS_DIR = '/content/drive/My Drive/Colab Notebooks/ObjectDetection/images/detected_objects/pass'

# try:
#   shutil.rmtree(BEDROOM_DIR)
#   shutil.rmtree(LIVING_DIR)
#   shutil.rmtree(KITCHEN_DIR)
#   shutil.rmtree(PASS_DIR)
# except:
#   os.makedirs(os.path.dirname(BEDROOM_DIR), exist_ok=True)
#   os.makedirs(os.path.dirname(LIVING_DIR), exist_ok=True)
#   os.makedirs(os.path.dirname(KITCHEN_DIR), exist_ok=True)
#   os.makedirs(os.path.dirname(PASS_DIR), exist_ok=True)
# finally:
#   os.makedirs(os.path.dirname(BEDROOM_DIR), exist_ok=True)
#   os.makedirs(os.path.dirname(LIVING_DIR), exist_ok=True)
#   os.makedirs(os.path.dirname(KITCHEN_DIR), exist_ok=True)
#   os.makedirs(os.path.dirname(PASS_DIR), exist_ok=True)

for file in os.listdir(IMG_FILE_DIR):
  image = cv2.imread(os.path.join(IMG_FILE_DIR,file))
#   image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  print(image.shape)
  image = imutils.resize(image,width=512)
    
  r = model.detect([image])[0]
#   image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
 
  fltrd_lbl = []

  for j in range(0,len(r["scores"])):
    classID = r["class_ids"][j]
    label = CLASS_NAMES[classID]
    color = [int(c) for c in np.array(COLORS[classID]) * 255]
    score = r["scores"][j]
    print(label, score)
    if(label.lower() == 'bed' or label.lower() == 'couch' or label.lower() == 'refrigerator' or label.lower() == 'sink' or label.lower() == 'oven' or label.lower() == 'microwave'):
      if(score >= 0.96):
        fltrd_lbl.append(label.lower())
        (startY,startX,endY,endX) = r["rois"][j]
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        text = "{}: {:.3f}".format(label, score)
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.putText(image, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX,0.6, color, 2)
  
#   from google.colab.patches import cv2_imshow
#   cv2_imshow(image)
  
  if('bed' in fltrd_lbl):
    print(file, "bedroom")
    cv2.imwrite(os.path.join(BEDROOM_DIR,file), image)
  elif('couch' in fltrd_lbl):
    print(file, "living")
    cv2.imwrite(os.path.join(LIVING_DIR,file), image)
  elif('sink' in fltrd_lbl or 'refrigerator' in fltrd_lbl or 'oven' in fltrd_lbl or 'microwave' in fltrd_lbl):
    print(file, "kitchen")
    cv2.imwrite(os.path.join(KITCHEN_DIR,file), image)
  else:
    print(file, "pass")
    cv2.imwrite(os.path.join(PASS_DIR,file), image)
  
  

(400, 320, 3)
bed 0.99913293
potted plant 0.9705955
vase 0.78708965
tufted_headboards_25.jpg bedroom
(3500, 3500, 3)
bed 0.99702954
tufted_headboards_27.jpg bedroom
(2000, 2000, 3)
bed 0.999114
potted plant 0.9747225
cup 0.78233296
vase 0.77782315
tufted_headboards_33.jpg bedroom
(523, 523, 3)
bed 0.9916112
clock 0.9536538
tufted_headboards_23.jpg bedroom
(600, 600, 3)
bed 0.998009
microwave 0.8620993
oven 0.8084577
vase 0.77732366
bowl 0.73765755
tufted_headboards_26.jpg bedroom
(776, 970, 3)
vase 0.9991429
bed 0.99255943
potted plant 0.9917903
bird 0.9914859
couch 0.9080098
couch 0.84458286
cup 0.74136
tufted_headboards_19.jpg bedroom
(501, 558, 3)
bed 0.99976975
tufted_headboards_24.jpg bedroom
(600, 800, 3)
bed 0.9972402
chair 0.9850133
chair 0.9810604
bed 0.82293373
vase 0.74101746
tufted_headboards_31.jpg bedroom
(646, 970, 3)
bed 0.9840152
chair 0.93121547
couch 0.70395935
tufted_headboards_32.jpg bedroom
(500, 500, 3)
couch 0.9674947
couch 0.8343123
tufted_headboards_18.jpg liv